In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import tensorflow as tf
from tensorflow.keras import backend as keras_backend
import numpy as np

gpu_on = True

if gpu_on :
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    gpu_devices = tf.config.experimental.list_physical_devices("CPU")

# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

print(gpu_devices)

[]


In [2]:
from src.data_loader.classification import ClassifyDataloader
from tensorflow.keras.applications.inception_v3 import preprocess_input
from glob import glob

task = "classification"
data_set_name = "detect_lvi"
batch_size = 32
on_memory = True
argumentation_proba = 0.8
target_size = (512, 512)
interpolation = "bilinear"
class_mode = "binary"
# class_mode = "categorical"
dtype="float32"


train_image_path_list = glob(f"./datasets/{task}/{data_set_name}/train/*/*")
valid_image_path_list = glob(f"./datasets/{task}/{data_set_name}/valid/*/*")
test_image_path_list = glob(f"./datasets/{task}/{data_set_name}/test/*/*")
label_list = os.listdir(f"./datasets/{task}/{data_set_name}/train")

label_to_index_dict = {label:index for index, label in enumerate(label_list)}
index_to_label_dict = {index:label for index, label in enumerate(label_list)}

train_data_loader = ClassifyDataloader(image_path_list=train_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=on_memory,
                                       argumentation_proba=argumentation_proba,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,
                                       shuffle=True,
                                       class_mode=class_mode,
                                       dtype=dtype
)
valid_data_loader = ClassifyDataloader(image_path_list=valid_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=on_memory,
                                       argumentation_proba=0,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)
test_data_loader = ClassifyDataloader(image_path_list=test_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=1,
                                       on_memory=False,
                                       argumentation_proba=0,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)

 [8608/8608] |##############################################| (Time:  0:07:10) 
 [03/1609] |                                                | (ETA:   0:01:11) 

Total data num 8608 with 2 classes


 [1609/1609] |##############################################| (Time:  0:01:17) 


Total data num 1609 with 2 classes
Total data num 1469 with 2 classes


In [3]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

import numpy as np
np.random.seed(1337)  # for reproducibility

DROPOUT_RATIO = 0.5

grad_cam = False
transfer_learning = False
epoch_release_frozen = 10
transfer_train_mode = "include_deep_layer"
layer_name_frozen_to = "mixed4"

#  binary_sigmoid, categorical_sigmoid, categorical_softmax
activation = "binary_sigmoid"

# create the base pre-trained model~
base_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(None,None,3),
    classes=None,
    pooling=None,
    classifier_activation=None
)

if transfer_learning:
    if train_mode == "dense_only":
        base_model.trainable = False
    elif train_mode == "include_deep_layer":
        for layer in base_model.layers: 
            layer.trainable = False
            if layer.name == layer_name_frozen_to:
                break

# add a global spatial average pooling layer
x = base_model.output
# (Batch_Size,?)
x = GlobalAveragePooling2D()(x)
x = Dropout(DROPOUT_RATIO)(x)
# let's add a fully-connected layer
# (Batch_Size,1)
x = Dense(1024, activation='relu')(x)
# (Batch_Size,1024)
x = Dropout(DROPOUT_RATIO)(x)

if grad_cam:
    x *= 1e-1
    keras_backend.set_floatx('float64')
    dense_dtype = "float64"
else:
    dense_dtype = "float32"
    
if activation == "binary_sigmoid":
    predictions = Dense(1, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = BinaryCrossentropy(label_smoothing=0.01)
elif activation == "categorical_sigmoid":
    predictions = Dense(2, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)
elif activation == "categorical_softmax":
    predictions = Dense(2, activation='softmax', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)

# this is the model we will train
model = Model(base_model.input, predictions)

In [4]:
from tensorflow.python.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.platform import tf_logging as logging

class ReduceLROnPlateauBestWeight(ReduceLROnPlateau):
    def __init__(self, *args, today_weight_path, load_frequently=False, **kwargs):
        super(ReduceLROnPlateauBestWeight, self).__init__(*args, **kwargs)
        self.today_weight_path = today_weight_path
        self.load_frequently= load_frequently
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            logging.warning('Reduce LR on plateau conditioned on metric `%s` '
                            'which is not available. Available metrics are: %s',
                             self.monitor, ','.join(list(logs.keys())))
        if not self.monitor_op(current, self.best): # not new best
            if not self.in_cooldown(): # and we're not in cooldown
                if self.load_frequently:
                    self.load_best_model()
                else:
                    if self.wait+1 >= self.patience: # going to reduce lr
                        self.load_best_model()
        super().on_epoch_end(epoch, logs) # actually reduce LR
        
    def load_best_model(self):
        weights = glob(f"{today_weight_path}/*.hdf5")
        weights.sort()
        best_path = weights[0]
        print(f"load weights: {best_path}")
        self.model.load_weights(best_path)


In [5]:
from datetime import date

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import Nadam

today = date.today()

# YY/MM/dd
today_str = today.strftime("%Y-%m-%d")
today_weight_path = f"./weights/{task}/{data_set_name}/{today_str}/target_size_{target_size}/" 
today_logs_path = f"./logs/{task}/{data_set_name}/{today_str}/target_size_{target_size}/"
os.makedirs(today_weight_path, exist_ok=True)
os.makedirs(today_logs_path, exist_ok=True)
optimizer = Nadam(1e-3, clipnorm=1)

save_c = ModelCheckpoint(
    today_weight_path+"/weights_{val_loss:.4f}_{loss:.4f}_{epoch:02d}.hdf5",
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min')


# reduceLROnPlat = ReduceLROnPlateau(
#     monitor="val_loss",
#     factor=0.1,
#     patience=20,
#     verbose=1,
#     mode="auto",
#     min_delta=0.0001,
#     cooldown=5,
#     min_lr=1e-7)
reduceLROnPlat = ReduceLROnPlateauBestWeight(
    today_weight_path=today_weight_path,
    load_frequently=True,
    monitor="val_loss",
    factor=0.1,
    patience=20,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=5,
    min_lr=1e-7)

csv_logger = CSVLogger(f'{today_logs_path}/log.csv', append=False, separator=',')


def make_model_trainable(epoch, model, target_epoch):
    if epoch > target_epoch:
        for layer in model.layers:
            layer.trainable = True
            
make_trainable_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: make_model_trainable(epoch, model, target_epoch=epoch_release_frozen)
)

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [6]:
start_epoch = 0
epochs = 200

model.fit(
    train_data_loader,
    validation_data=valid_data_loader,
    epochs=epochs,
    callbacks=[reduceLROnPlat, save_c, csv_logger, make_trainable_callback],
    initial_epoch=start_epoch
)

Epoch 1/200
269/269 [==============================] - 286s 981ms/step - loss: 0.6898 - accuracy: 0.6200 - val_loss: 1.5831 - val_accuracy: 0.5556
Epoch 2/200
269/269 [==============================] - 264s 980ms/step - loss: 0.6282 - accuracy: 0.6655 - val_loss: 0.5675 - val_accuracy: 0.7387
Epoch 3/200
269/269 [==============================] - 265s 983ms/step - loss: 0.5912 - accuracy: 0.6948 - val_loss: 4.0129 - val_accuracy: 0.5569
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.4380_0.3824_02.hdf5
Epoch 4/200
269/269 [==============================] - 265s 982ms/step - loss: 0.4591 - accuracy: 0.7829 - val_loss: 0.7305 - val_accuracy: 0.8156
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.4380_0.3824_02.hdf5
Epoch 5/200
269/269 [==============================] - 264s 979ms/step - loss: 0.3782 - accuracy: 0.8262 - val_loss: 1.0706 - val_accuracy: 0.7887
load weights: ./weights/classificatio

load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5

Epoch 00034: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 35/200
269/269 [==============================] - 264s 980ms/step - loss: 0.1863 - accuracy: 0.9074 - val_loss: 0.2403 - val_accuracy: 0.9287
Epoch 36/200
269/269 [==============================] - 264s 979ms/step - loss: 0.1843 - accuracy: 0.9044 - val_loss: 0.3095 - val_accuracy: 0.9006
Epoch 37/200
269/269 [==============================] - 264s 979ms/step - loss: 0.1740 - accuracy: 0.9144 - val_loss: 0.3494 - val_accuracy: 0.8906
Epoch 38/200
269/269 [==============================] - 264s 980ms/step - loss: 0.1725 - accuracy: 0.9133 - val_loss: 0.3375 - val_accuracy: 0.8956
Epoch 39/200
269/269 [==============================] - 263s 978ms/step - loss: 0.1729 - accuracy: 0.9133 - val_loss: 0.4135 - val_accuracy: 0.8838
Epoch 40/200
269/269 [==============================] - 264s 979

load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5
Epoch 70/200
269/269 [==============================] - 264s 979ms/step - loss: 0.1957 - accuracy: 0.9018 - val_loss: 0.1960 - val_accuracy: 0.9406
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5
Epoch 71/200
269/269 [==============================] - 264s 979ms/step - loss: 0.2006 - accuracy: 0.9046 - val_loss: 0.1951 - val_accuracy: 0.9406
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5
Epoch 72/200
269/269 [==============================] - 265s 981ms/step - loss: 0.2036 - accuracy: 0.8977 - val_loss: 0.1955 - val_accuracy: 0.9400
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5
Epoch 73/200
269/269 [==============================] - 264s 981ms/step - loss: 0.2049 - accuracy: 0

Epoch 103/200
269/269 [==============================] - 265s 981ms/step - loss: 0.2045 - accuracy: 0.8975 - val_loss: 0.1897 - val_accuracy: 0.9388
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5
Epoch 104/200
269/269 [==============================] - 264s 980ms/step - loss: 0.2046 - accuracy: 0.8989 - val_loss: 0.1871 - val_accuracy: 0.9400
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5
Epoch 105/200
269/269 [==============================] - 264s 981ms/step - loss: 0.2036 - accuracy: 0.8973 - val_loss: 0.1876 - val_accuracy: 0.9400
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5
Epoch 106/200
269/269 [==============================] - 264s 980ms/step - loss: 0.2045 - accuracy: 0.9013 - val_loss: 0.1886 - val_accuracy: 0.9388
load weights: ./weights/classification/detect_lvi/2021-07-01/t

Epoch 138/200
269/269 [==============================] - 264s 980ms/step - loss: 0.2018 - accuracy: 0.9014 - val_loss: 0.1867 - val_accuracy: 0.9406
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5
Epoch 139/200
269/269 [==============================] - 264s 981ms/step - loss: 0.2102 - accuracy: 0.8949 - val_loss: 0.1877 - val_accuracy: 0.9394
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5
Epoch 140/200
269/269 [==============================] - 265s 982ms/step - loss: 0.2022 - accuracy: 0.8974 - val_loss: 0.1883 - val_accuracy: 0.9381
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5
Epoch 141/200
269/269 [==============================] - 265s 981ms/step - loss: 0.2005 - accuracy: 0.9087 - val_loss: 0.1904 - val_accuracy: 0.9388
load weights: ./weights/classification/detect_lvi/2021-07-01/t

269/269 [==============================] - 265s 981ms/step - loss: 0.2001 - accuracy: 0.8989 - val_loss: 0.1879 - val_accuracy: 0.9388
load weights: ./weights/classification/detect_lvi/2021-07-01/target_size_(512, 512)/weights_0.1769_0.2184_14.hdf5


In [7]:
target_data_loader = test_data_loader

target_data_loader.batch_size = 1
# model.load_weights('./weights/classification/detect_lvi/2021/06/25/'+"weights_26_0.0724.hdf5")
model_predicted = model.predict(target_data_loader)
ground_truth = np.array([label for img_array, label in target_data_loader])

model.evaluate(target_data_loader)

1469/1469 [==============================] - 70s 48ms/step - loss: 0.0445 - accuracy: 0.9952


[0.04447188600897789, 0.9952348470687866]

In [7]:
negative_group_1 = np.where(model_predicted < 0.5)[0]
negative_group_2 = np.where(model_predicted != 0)[0]
negative_confirm = np.where(model_predicted == 0)[0]
negative_not_confirm = np.intersect1d(negative_group_1, negative_group_2)

positive_group_1 = np.where(model_predicted > 0.5)[0]
positive_group_2 = np.where(model_predicted != 1)[0]
postive_confirm = np.where(model_predicted == 1)[0]
postive_not_confirm = np.intersect1d(positive_group_1, positive_group_2)

ground_truth_negative = np.where(ground_truth == 0)[0]
ground_truth_positive = np.where(ground_truth == 1)[0]

true_negative = np.intersect1d(negative_group_1, ground_truth_negative)
true_positive = np.intersect1d(positive_group_1, ground_truth_positive)
false_negative = np.intersect1d(negative_group_1, ground_truth_positive)
false_positive = np.intersect1d(positive_group_1, ground_truth_negative)


In [8]:
print(len(negative_confirm))
print(len(negative_not_confirm))
print(len(postive_confirm))
print(len(postive_not_confirm))

0
813
0
656


In [9]:
print(len(negative_confirm))
print(len(negative_not_confirm))
print(len(postive_confirm))
print(len(postive_not_confirm))

0
813
0
656


In [11]:
print(false_negative)
print(false_positive)

[ 812  813  823  824  831 1049 1054 1055 1056 1057 1058 1059 1062 1063
 1065 1067 1068 1070 1072 1073 1074 1075 1076 1078 1079 1080 1082 1083
 1084 1085 1086 1232 1234 1235 1239 1240 1241 1252 1263 1274 1285 1389
 1394 1395 1400 1451 1456 1461 1463]
[  51  446  471  489  629 1468]


In [13]:
test_data_loader.data_getter.image_path_dict[1468]

'./datasets/classification/detect_lvi/test/LVI_positive/211_LVI_positive_9.png'

In [12]:
test_data_loader.data_getter.class_dict[1468]

0